<a href="https://colab.research.google.com/github/albertorock82/practicasgit/blob/main/obtenerGruposdeKW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet gspread pandas unidecode nltk


In [11]:
# Paso 1: Instalar dependencias
!pip install --quiet gspread pandas unidecode spacy
!python -m spacy download es_core_news_sm

import pandas as pd
import gspread
from unidecode import unidecode
import spacy
from collections import defaultdict
import re
import string

from google.colab import auth
from google.oauth2.service_account import Credentials

# --- PARÁMETROS A EDITAR ---
NOMBRE_ARCHIVO_JSON = '/content/ambient-antenna-440101-u4-d9758bb09208.json'  # Sube tu clave JSON y pon aquí el nombre
NOMBRE_LIBRO = 'KeywordDepliadorKS_serp'
NOMBRE_HOJA = 'Sheet1'  # hoja donde están las keywords en columna A
NOMBRE_HOJA_RESULTADO = 'grupos'

# --- AUTENTICACIÓN CON GOOGLE SHEETS ---
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

credentials = Credentials.from_service_account_file(NOMBRE_ARCHIVO_JSON, scopes=scope)
gc = gspread.authorize(credentials)

# --- CARGAR DATOS ---
sh = gc.open(NOMBRE_LIBRO)
worksheet = sh.worksheet(NOMBRE_HOJA)
keywords = worksheet.col_values(1)  # Columna A completa

# --- PREPROCESAMIENTO ---
nlp = spacy.load("es_core_news_sm")

agrupaciones = defaultdict(set)

def es_valida(palabra):
    # Descarta palabras muy cortas, solo numéricas, o vacías
    return (
        len(palabra) >= 3 and
        not re.match(r'^\d+$', palabra) and
        palabra not in {'como', 'es', 'de', 'los', 'las', 'para', 'el', 'la',
                        'por', 'con', 'un', 'una', 'al', 'en', 'y', 'o'}
    )

for fila in keywords:
    # Elimina puntuación, tildes, pasa a minúsculas
    fila = unidecode(fila.lower())
    fila = fila.translate(str.maketrans('', '', string.punctuation))
    palabras = fila.split()

    for palabra in palabras:
        if es_valida(palabra):
            doc = nlp(palabra)
            if doc:
                lema = doc[0].lemma_
                agrupaciones[lema].add(palabra)

# --- PREPARAR RESULTADO ---
df_grupos = pd.DataFrame([
    {"identificador": clave, "variantes": ", ".join(sorted(list(valores)))}
    for clave, valores in agrupaciones.items()
])

# --- CREAR NUEVA HOJA Y ESCRIBIR ---
# Borrar hoja si ya existe
try:
    sh.del_worksheet(sh.worksheet(NOMBRE_HOJA_RESULTADO))
except:
    pass

ws_out = sh.add_worksheet(title=NOMBRE_HOJA_RESULTADO, rows=len(df_grupos)+1, cols=2)
ws_out.update([["identificador", "variantes"]] + df_grupos.values.tolist())

print("✅ Agrupación semántica completada. Revisa la hoja 'agrupaciones'")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 46.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Agrupación semántica completada. Revisa la hoja 'agrupaciones'
